## Extracting data from ESRI Map Servers
More and more organizations are hosting their data on ESRI Map Servers, giving us access to these datasets via "ArcGIS REST Service Endpoints". For example, NC OneMap provides access to a wealth of data here:https://services.nconemap.gov/secure/rest/services. This means there's a huge wealth of data we can access - either to download locally or use directly - if only we know how to access it. And to do that, we need to learn the [ESRI REST API](https://developers.arcgis.com/rest/services-reference/get-started-with-the-services-directory.htm). 

In this notebook, we explore two particularly useful web services provided in the ESRI REST API: The `Export Map` service applied to a `MapServer` service, and the `Query` service applied to a specific `Feature Layer` within a `MapService`. (What those are will be clear as we move along in this exercise...)

We will examine these web services first from with a web browser, which is a handy way to explore what's what, and then we'll convey this knowledge of the API to the Python scripting environment, which is a much more powerful means for tapping into the API. 

By no means is this an exhaustive look at the ESRI REST API, but it should give you a good foundation for seeing how these web services work, both in a web browser and within Python so that you can continue to learn more on your own.

### 1. Navigating the Service Page
* Open up the NC OneMap main services page: https://services.nconemap.gov/secure/rest/services
* Note the organization: You see a set of **folders** followed by a list of **services**. 
 * The <u>folders</u> simply provide a hierarchical organization as they contain either more folders or more services. 
 * The <u>services</u> list the access points to various datasets hosted. You'll see a variety of different **service types**. The main map service types that provide access to data include:
  * [`MapServer`](https://developers.arcgis.com/rest/services-reference/map-service.htm)services offer access to the contents of a map hosted on a server.
  * [`FeatureServer`](https://developers.arcgis.com/rest/services-reference/feature-service.htm)services contain datasets (for example, tables and views) with or without a spatial column.
  * [`ImageServer`](https://developers.arcgis.com/rest/services-reference/image-service.htm) services provides access to raster data.

* Click on the [`NC1Map_Boundaries (MapServer)`](https://services.nconemap.gov/secure/rest/services/NC1Map_Boundaries/MapServer) link to expose information on that map service. 
 * Familiarize yourself with the information listed here. See that this MapService includes two layers: `Townships` and `Zip Code Tabulation Areas`, and they are referenced by the indices provided - `0` and `1`, respectively. 
 * At the bottom of the page, you'll see a list of supporting services: `Export Map`,`Identify`,... What each does is described in the [ESRI documentation](https://developers.arcgis.com/rest/services-reference/map-service.htm#ESRI_SECTION1_FD2E4C48DA0A4D398909699F1F83CE2F). We'll be looking at the [Export Map](https://developers.arcgis.com/rest/services-reference/export-map.htm) service next. 
 * But first, click on the [`View In: ArcGIS JavaScript`](https://services.nconemap.gov/secure/rest/services/NC1Map_Boundaries/MapServer?f=jsapi) to get a quick glimps of how the data look. 
 * You may also want to view the ['Legend'](https://services.nconemap.gov/secure/rest/services/NC1Map_Boundaries/MapServer/legend) and other link on the main service page. 


### 2. Exploring the `Export Map` service
The Export Map service produces a figure of the spatial data with constraints we give it. These constraints can include: a <u>zoom</u> to a particular region, showing only <u>specific layers</u> listed in the map service, and showing only <u>specific features</u> in a given layer. Here, we explore how to do that using the NC OneMap's Census Map Service. 

* Go back to the NC OneMap main services page: https://services.nconemap.gov/secure/rest/services
* We'll now examine the [`NC1Map Census Map Service`](https://services.nconemap.gov/secure/rest/services/NC1Map_Census/MapServer).
 * → *How many layers in this map service?*
* Click on the [`Export Map`](https://services.nconemap.gov/secure/rest/services/NC1Map_Census/MapServer/export?bbox=115398.88167874969,137561.22299815627,951284.8934306827,392245.24220382335) link at the bottom of the MapServer's main page. This will open a page where we can tinker with all the parameters related to the service.
 * Change the zoom of the map by supplying the coordinates `-78.8, 35.7, -79.0, 36.3` in the `Bounding Box:` setting. 
 * Because the bounding coordinates we just gave are in WGS 84, set the `Bounding Box Spatial Reference` to the [WKID](http://spatialreference.org/ref/epsg/wgs-84/) for WGS 84, which happens to be `4326`.
 * Refresh the map by hitting `Export Map (Get)` at the bottom to see our changes (or just `Enter` might work).

→ *We just zoomed our map using bounding box coordinates!* Your map should be zoomed in to an area around Durham County.

Now to show specific layers.

 * In the `Layers:` box, enter `show:0` to display only the first layer in the map service (1970 Census Boundary/Population ). Then click `Export Map (Get)` at the bottom. Refresh your map and just those data should appear. 
 * Change the value so just Layer 1, the 1980 Census Boundary/Population layer, appears. 
 * Try once more, this time showing Layer 2, 1999 Census block groups,. *Nothing appears! Why? Because that layer, which shows fine scale data, is set not too appear unless we zoom in more. How might you do that?*
 * Try entering `show:6,1` in the Layers box... 

Finally, let's filter what records are shown. Here we'll select features in Layer 9 (2010 Tracts) that have a GEOID10 starting with "37063", which is the FIPS code for Durham County.
 * Set the `Layers` box to show Layer 9. 
 * In the `Layer Definitions` box, enter `{"9":"GEOID10 LIKE '37063%'"}`
 * The FIPS for Orange County is 37135. Can you display only Orange County tracts?

---
### 3. Using the Export Map Service from Python
Now that we have a feel for 

In [2]:
import requests

In [24]:
#Get the URL of the map service for layer 9 in the NC1Map Census map service folder
serviceURL = 'https://services.nconemap.gov/secure/rest/services/NC1Map_Census/MapServer/export'

In [25]:
#Set the parameters for our service
params = {'bbox':'-78.8%2C+35.7%2C+-79.0%2C+36.3',
           'bboxSR':'4326',
           'layers':'',
           'layerDefs':'%7B%229%22%3A%22GEOID10+LIKE+%2737063%25%27%22%7D',
           'size':'',
           'imageSR':'',
           'format':'png',
           'transparent':'false',
           'dpi':'',
           'time':'',
           'layerTimeOptions':'',
           'dynamicLayers':'',
           'gdbVersion':'',
           'mapScale':'',
           'rotation':'',
           'datumTransformations':'',
           'layerParameterValues':'',
           'mapRangeValues':'',
           'layerRangeValues':'',
           'f':'json'}

In [26]:
#Query the service url, in JSON format
response = requests.get(serviceURL,params)
json_response = response.json()

In [27]:
json_response.keys()

dict_keys(['href', 'width', 'height', 'extent', 'scale'])

In [29]:
imageURL = json_response['href']

In [32]:
from IPython.display import Image
Image(url=imageURL)